In [1]:
!pip install langchain kor

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 22.3 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pandas: filename=pandas-1.5.3-cp312-cp312-macosx_10_9_universal2.whl size=17005332 sha256=af0c9cf14c295427fbb17f2441faa9ebf6a43096fb28709e9b1db90dbfb5bc76
  Stored in directory: /Users/jity/Library/Caches/pip/wheels/fb/83/18/8e7307aa1185c5498c5490e4d9c8a1732d9f1056e86c3491c6
Successfully built pandas
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2


In [6]:
from langchain.chat_models import ChatOpenAI
from kor import create_extraction_chain, Object, Text

In [7]:
import os
import getpass

In [10]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()

 ········


In [15]:
llm = ChatOpenAI(model_name="gpt-4o")

In [16]:
schema = Object(
    id="task manager",
    description="User is specifying what changes to apply to their calendar",
    attributes=[
        Text(
            id="action",
            description="This is the the verb associated with the change the user wants to apply to their calendar. Most common examples are creating, deleting, updating, viewing the user's scehdule.",
            many=True,
            examples=[
                ("Schedule a meeting with the marketing team for next Monday at 10 AM.", "Create"),
                ("Change my meeting with John on Thursday to Friday at 2 PM.", "Update"),
                ("Cancel my lunch meeting with Sarah tomorrow.", "Delete"),
                ("Show me my calendar for the week.", "View"),
                ("When am I free this week for a 2-hour meeting?", "View"),
                ("Block off time for focused work from 9 AM to 12 PM every day.", "Create"),
                ("Reserve the next Saturday afternoon for personal time.", "Create"),
                ("Find a time when I can schedule a workout session.", "View"),
            ]
        ),
        Text(
            id="time/date",
            description="This is the time and date of the event or new change that the user wants to apply.",
            many=True,
            examples=[
                ("Schedule a meeting with the marketing team for next Monday at 10 AM.", "Monday, 10 AM"),
                ("Change my meeting with John on Thursday to Friday at 2 PM.", "Friday, 2 PM"),
                ("Cancel my lunch meeting with Sarah tomorrow.", "Tomorrow"),
                ("Block off time for focused work from 9 AM to 12 PM every day.", "Every day, 9 AM to 12 PM"),
                ("Reserve the next Saturday afternoon for personal time.", "Saturday, afternoon"),
                ("Find a time when I can schedule a workout session.", ""),
            ]
        ),
        Text(
            id="event",
            description="This is a short description of what the user would be doing (or would have been doing if the action was to cancel) in the specified time/date.",
            many=True,
            examples=[
                ("Schedule a meeting with the marketing team for next Monday at 10 AM.", "Meeting with the marketing team"),
                ("Change my meeting with John on Thursday to Friday at 2 PM.", "Meeting with John"),
                ("Cancel my lunch meeting with Sarah tomorrow.", "Meeting with Sarah"),
                ("Block off time for focused work from 9 AM to 12 PM every day.", "Focused work"),
                ("Reserve the next Saturday afternoon for personal time.", "Personal time"),
                ("Find a time when I can schedule a workout session.", "Workout session"),
            ]
        )
    ]
)

In [17]:
chain = create_extraction_chain(llm, schema, encoder_or_encoder_class="json")

In [21]:
chain.invoke("Create a task to 'Review project proposal' due by end of the day.")["text"]["data"]

{'task manager': {'action': ['Create'],
  'time/date': ['End of the day'],
  'event': ['Review project proposal']}}